# A Tiny Temperature Converter Library
## Written in Julia (a naive approach)

Based on a wonderful example from _Erik Engheim_

We start with some type definitions!

In [ ]:
abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end

struct Kelvin <: Temperature
    value::Float64 
end

struct Fahrenheit <: Temperature
    value::Float64
end

Define the promotion rules, so Julia knows our preferences when mixing types.

In [ ]:
import Base: promote_rule  # we import the `promote_rule` function to add our own methods

promote_rule(::Type{Kelvin}, ::Type{Celsius})     = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin})  = Kelvin

We implement the conversion logic by adding methods to the `convert` function, available in the `Base` of Julia.

In [ ]:
import Base: convert  # again, we add our own methods to `convert`

convert(::Type{Kelvin},  t::Celsius)     = Kelvin(t.value + 273.15)
convert(::Type{Kelvin},  t::Fahrenheit)  = Kelvin(Celsius(t))
convert(::Type{Celsius}, t::Kelvin)      = Celsius(t.value - 273.15)
convert(::Type{Celsius}, t::Fahrenheit)  = Celsius((t.value - 32)*5/9)
convert(::Type{Fahrenheit}, t::Celsius)  = Fahrenheit(t.value*9/5 + 32)
convert(::Type{Fahrenheit}, t::Kelvin)   = Fahrenheit(Celsius(t))

We can add some nice constructors, so we can initialise each 

In [ ]:
Kelvin(t::Temperature) = convert(Kelvin, t)
Celsius(t::Temperature) = convert(Celsius, t)
Fahrenheit(t::Temperature) = convert(Fahrenheit, t)

In [ ]:
Kelvin(Fahrenheit(23))

In [ ]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

Adding arithmetic operations for our types by extending the `Base` operators.

In [ ]:
import Base: +, -, *  # operators are functions, we can extend them easily!

+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)

+(x::T, y::T) where {T <: Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T <: Temperature} = T(x.value - y.value)

*(x::Number, y::T) where {T <: Temperature} = T(x * y.value);

### Let's test it:

In [ ]:
Celsius(37) + Kelvin(10)

In [ ]:
Celsius(Celsius(37) + Kelvin(10))

In [ ]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

### A nice way to create some syntactic sugar:

In [ ]:
const °C = Celsius(1)
const °F = Fahrenheit(1)
const K = Kelvin(1);


In [ ]:
5°F, 23K, 42°C

In [ ]:
42°C + 10K

### Alright, let's have a look behind the scenes...

In [ ]:
example_calculation() = Fahrenheit(2) + Celsius(3) + Kelvin(4)

In [ ]:
@code_llvm example_calculation()

#### LLVM figured out that the function returns always the same value... Dooh ;)

### This is the "machine code":

In [ ]:
@code_native example_calculation()

### Now a function which is not constant ;)

In [ ]:
another_calculation(a, b, c) = Fahrenheit(a) + Celsius(b) + Kelvin(c)

In [ ]:
@code_llvm debuginfo=:none another_calculation(1, 2, 3)

### And the machine code:

In [ ]:
@code_native debuginfo=:none another_calculation(2.0, 3.0, 4.0)

## What about the actual performance?

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark another_calculation(temperatures...) setup=(temperatures=rand(3))

### The power of multiple dispatch

We can easily define our own `rand` method for `Kelvin` temperatures ()

In [ ]:
using Random
Random.rand(rng::AbstractRNG, ::Random.SamplerType{Kelvin}) =  Kelvin(rand() * 5000)

In [ ]:
rand(Kelvin)

In [ ]:
temperatures = rand(Kelvin, 1_000_000)

### Is this fast?

In [ ]:
@benchmark rand(Kelvin, 1_000_000)

In [ ]:
@benchmark rand(1_000_000)

### Yep... 2ms, 2 allocs, 8 MiB memory

### Convert them to `Celsius` by using our type constructor and the `.`-notation for element-wise operation (similar to Matlab or `ufuncs` in numpy)

In [ ]:
Celsius.(temperatures)

### Is this fast?

In [ ]:
@benchmark Celsius.($temperatures)

### Yep... 1.5ms, 2 allocs, 8 MiB memory

## And we magically get all the other methods of the `rand()` function:

In [ ]:
rand(Kelvin, 100, 200, 300)